In [14]:
import rasterio
import numpy as np

# Define your value_to_value dictionary
value_to_value = {
    11: 0.0,
    12: 0.0,
    21: 0.0,#39
    22: 148.41,
    23: 0.0,
    #missing 24 high intensity
    24: 0.0,
    31: 0.0,
    41: 0.0,
    42: 0.0,
    43: 0.0,
    44: 0.0,# what are these
    45: 0.0,# what are these
    46: 0.0,
    51: 0.0,
    52: 0.0,  
    71: 0.0,
    73: 0.0,
    74: 0.0,
    81: 0.0,
    82: 0.0,
    90: 0.0,
    95: 0.0
}

# Open the input TIFF file
with rasterio.open('/shared_space/ACV/nlcd_nc0.1.tif') as src:
    # Read the image data into a numpy array
    data = src.read(1)  # Assuming single band, modify if you have multiple bands

    # Replace the values according to the dictionary
    for key, value in value_to_value.items():
        data[data == key] = value

    # Define the output file path
    output_file = 'only_lowest_22_output.tif'

    # Write the modified data to a new TIFF file
    with rasterio.open(output_file, 'w', 
                       driver='GTiff', 
                       count=1, 
                       dtype=data.dtype, 
                       crs=src.crs, 
                       transform=src.transform, 
                       width=src.width, 
                       height=src.height) as dst:
        dst.write(data, 1)

print(f"Modified TIFF saved as {output_file}")

Modified TIFF saved as only_lowest_21_output.tif


In [11]:
import rasterio
import numpy as np
import geopandas as gpd
import rasterio.mask
import matplotlib.pyplot as plt

# Function to calculate statistics for a given county
def calculate_stats(county, src):
    # Mask the raster data using the county polygon
    geometry = [county['geometry']]
    out_image , out_transform = rasterio.mask.mask(src, geometry, crop=True)
    
    # Remove masked NoData values (assuming NoData is -99999)
    masked_data = np.ma.masked_equal(out_image, -99999)
    
    # Calculate mean, standard deviation, and maximum values
    mean_cost = np.mean(masked_data)
    max_cost = np.max(masked_data)
    std_cost = np.std(masked_data)
    
    return mean_cost, max_cost, std_cost

# Assuming 'output.tif' is the modified raster after applying the value_to_value mapping
output_file = 'only_lowest_21_output.tif'

# Open the modified raster file (output.tif)
with rasterio.open(output_file) as src:
    # List of counties to analyze (replace with your actual GeoDataFrame or shapefile of counties)
    counties = gpd.read_file('/data/HW/HW2/NCDOT_County_Boundaries.shp')  # Replace with your shapefile
    counties_reprojected = counties.to_crs(epsg=4326)
    results = []
    
    # Calculate statistics for each county
    for idx, county in counties_reprojected.iterrows():
        mean_cost, max_cost, std_cost = calculate_stats(county, src)
        results.append({
            'County': county['CountyName'],
            'Percent Land Developed Open Space': mean_cost,
            'Max Cost': max_cost,
            'Std Cost': std_cost
        })
    
    # Create a GeoDataFrame from the results
    results_gdf = gpd.GeoDataFrame(results)
    
    # Print or visualize the results
    print(results_gdf)


         County  Max Cost  Percent Land Developed Open Space   Std Cost
0        Camden        39                           0.194030   2.743996
1         Gates        39                           0.906318   5.875797
2       Iredell        39                           2.753695   9.990558
3        Wilkes        39                           1.400756   7.257229
4         Union        39                           2.387069   9.348668
5      Cabarrus        39                           3.001818  10.395191
6          Wake        39                           3.785965  11.546389
7      Franklin        39                           0.971146   6.077136
8        Pender        39                           0.754113   5.370447
9   New Hanover        39                           1.659574   7.872053
10       Durham        39                           4.633971  12.619477
11       Orange        39                           3.343750  10.919047
12      Chatham        39                           1.676904   7

In [13]:
# Save DataFrame to Excel file
results_gdf.to_csv('2020_land_developed_open_space.csv')